In [11]:
pip install scikit-learn


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.8 kB 2.6 MB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.2/11.0 MB 25.5 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/11.0 MB 32.8 MB/s eta 0:00:01
   -------------- ------------------------- 4.1/11.0 MB 29.2 MB/s eta 0:00:01
   ----------------- ---------------------- 4.8/11.0 MB 25.7 MB/s eta 0:00:01
   ------------------------ --------------- 6.8/11.0 MB 29.1 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 31.0 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.0 MB 36.4 MB/s eta 0:00:01
   ----------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
pip install xgboost

  Using cached xgboost-2.1.2-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-2.1.2-py3-none-win_amd64.whl (124.9 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [13]:
df = pd.read_csv('C:/Users/Admin/Desktop/수업자료/vs code/musinsa_size_XGB.csv', encoding='utf-8-sig')

###데이터 전처리

In [14]:
# '키'와 '몸무게'에서 숫자 부분만 추출 후 숫자형으로 변환
df['height'] = df['height'].astype(str).str.extract(r'(\d+)').astype(float)
df['weight'] = df['weight'].astype(str).str.extract(r'(\d+)').astype(float)

# 결과 확인
print(df[['height', 'weight']].head())


   height  weight
0   177.0    85.0
1   164.0    50.0
2   159.0    50.0
3   183.0    68.0
4   177.0    77.0


In [15]:
# '사이즈'를 문자열로 변환 후 처리
df['size'] = df['size'].astype(str).str.replace(r'[^SMLX23]', '', regex=True)

# '2XL'을 'XL'로 대체
df.loc[df['size'] == '2XL', 'size'] = 'XL'

# 'XS'을 'S'로 대체
df.loc[df['size'] == 'XS', 'size'] = 'S'

# 원하는 사이즈 값만 남기기
df = df[df['size'].isin(['M', 'L', 'S', 'XL'])]

# 결과 확인
print(df['size'])

1         L
2         S
3         L
4         M
5         M
         ..
75493     L
75494     L
75495     L
75496     S
75497    XL
Name: size, Length: 68688, dtype: object


In [16]:
# #복사본 저장해두기
df2 =df.copy()

In [17]:
df['size'].value_counts()

size
L     24664
M     19831
XL    18065
S      6128
Name: count, dtype: int64

XGB
- MIN-MAX

In [18]:
df=df2.copy()

In [19]:
# BMI 범주화하여 새로운 컬럼 'bmi_category' 추가
def categorize_bmi(bmi):
    if bmi < 18.5:
        return '저체중'
    elif 18.5 <= bmi < 24.9:
        return '정상'
    else:
        return '과체중'

In [22]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from xgboost import XGBClassifier
import numpy as np

# 데이터 준비 및 전처리 (이전과 동일)
size_order = ['S', 'M', 'L', 'XL']
df['size'] = pd.Categorical(df['size'], categories=size_order, ordered=True)
df['size'] = df['size'].cat.codes

label_enc_gender = LabelEncoder()
df['gender'] = label_enc_gender.fit_transform(df['gender'])

df['BMI'] = df['weight'] / (df['height'] / 100) ** 2

df['bmi_category'] = df['BMI'].apply(categorize_bmi)
bmi_category_mapping = {'저체중': 0, '정상': 1, '과체중': 2}
df['bmi_category'] = df['bmi_category'].map(bmi_category_mapping)

X = df[['height', 'weight', 'bmi_category', 'gender']]
y = df['size']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X[['height', 'weight']])
X_scaled = pd.DataFrame(X_scaled, columns=['height', 'weight'])
X_scaled['gender'] = X['gender'].values
X_scaled['bmi_category'] = X['bmi_category'].values

# 최적의 하이퍼파라미터를 적용한 XGBoost 모델
best_params = {'colsample_bytree': 1.0, 'learning_rate': 0.3, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 50, 'subsample': 0.7}
xgb_final_model = XGBClassifier(**best_params, use_label_encoder=False)
xgb_final_model.fit(X_scaled, y)  # 전체 데이터로 학습

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:48:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [29]:
# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 최적의 하이퍼파라미터로 학습
xgb_final_model = XGBClassifier(**best_params, use_label_encoder=False)
xgb_final_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = xgb_final_model.predict(X_test)

# 정확도 및 분류 보고서 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"모델 정확도: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=size_order))


모델 정확도: 0.6120
Classification Report:
               precision    recall  f1-score   support

           S       0.53      0.41      0.46      1199
           M       0.58      0.63      0.60      4050
           L       0.58      0.60      0.59      4944
          XL       0.72      0.68      0.70      3545

    accuracy                           0.61     13738
   macro avg       0.60      0.58      0.59     13738
weighted avg       0.61      0.61      0.61     13738



c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:55:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


예측 함수 설정

In [ ]:
##함수 정의 

def predict_size(height, weight, gender):
    # BMI 계산
    bmi = weight / (height / 100) ** 2
    bmi_category_label = categorize_bmi(bmi)
    
    # BMI 범주를 숫자로 변환
    bmi_category_mapping = {'저체중': 0, '정상': 1, '과체중': 2}
    bmi_category = bmi_category_mapping[bmi_category_label]
    
    # 입력 데이터 생성
    input_data = pd.DataFrame([[height, weight, gender, bmi_category]], columns=['height', 'weight', 'gender', 'bmi_category'])
    input_data[['height', 'weight']] = scaler.transform(input_data[['height', 'weight']])  # 스케일링
    
    # 열 순서를 모델 학습 시 사용한 순서와 동일하게 변경
    input_data = input_data[['height', 'weight', 'gender', 'bmi_category']]
    
    # 예측
    prediction = xgb_final_model.predict(input_data)
    
    # 사이즈를 원래 레이블로 변환
    predicted_size = pd.Categorical.from_codes(prediction, categories=size_order)
    return predicted_size[0]

In [28]:
# 예시 입력값으로 예측
example_prediction = predict_size(175, 70, 0)  # 키: 175cm, 몸무게: 70kg, 성별: 남성(0)
print("예측된 사이즈:", example_prediction)

예측된 사이즈: L
